##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 分布式输入

<table class="tfo-notebook-buttons" align="left">
  <td data-segment-approved="false"><a target="_blank" href="https://tensorflow.google.cn/tutorials/distribute/input"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a></td>
  <td data-segment-approved="false"><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/distribute/input.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
  <td data-segment-approved="false"><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/distribute/input.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a></td>
  <td data-segment-approved="false"><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tutorials/distribute/input.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
</table>

[tf.distribute](https://tensorflow.google.cn/guide/distributed_training) API 为用户提供了一种简单的方法，可将训练范围从一台计算机扩展到多台计算机。扩展模型时，用户还必须将其输入分布到多个设备上。`tf.distribute` 提供了相应的 API，您可以利用这些 API 在设备之间自动分布输入。

本指南将展示使用 `tf.distribute` API 创建分布式数据集和迭代器的不同方法。此外，还将涵盖以下主题：

- 使用 `tf.distribute.Strategy.experimental_distribute_dataset` 和 `tf.distribute.Strategy.distribute_datasets_from_function` 时的用法、分片和批处理选项。
- 遍历分布式数据集的不同方式。
- `tf.distribute.Strategy.experimental_distribute_dataset`/`tf.distribute.Strategy.distribute_datasets_from_function` API 和 `tf.data` API 之间的区别以及用户在使用中可能遇到的任何限制。

本指南不介绍如何将分布式输入与 Keras API 一起使用。

## 分布式数据集

要使用 `tf.distribute` API 调整范围，建议用户使用 `tf.data.Dataset` 表示其输入。`tf.distribute` 已经可与 `tf.data.Dataset` 高效地协同工作（例如，将数据自动预提取到每个加速器设备上），同时会定期将性能优化合并到实现中。如果您有使用除 `tf.data.Dataset` 以外的 API 的用例，请参阅本指南后面的[部分](%22tensorinputs%22)。在非分布式训练循环中，用户首先创建一个 `tf.data.Dataset` 实例，然后迭代各个元素。例如：


In [ ]:
import tensorflow as tf

# Helper libraries
import numpy as np
import os

print(tf.__version__)

In [ ]:
global_batch_size = 16
# Create a tf.data.Dataset object.
dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(100).batch(global_batch_size)

@tf.function
def train_step(inputs):
  features, labels = inputs
  return labels - 0.3 * features

# Iterate over the dataset using the for..in construct.
for inputs in dataset:
  print(train_step(inputs))


为了在尽可能不更改用户现有代码的情况下使用户能够使用 `tf.distribute` 策略，我们引入了两个 API，它们将分配 `tf.data.Dataset` 实例并返回一个分布式数据集对象。随后，用户可以遍历此分布式数据集实例并像以前一样训练自己的模型。现在让我们更详细地看一下这两个 API - `tf.distribute.Strategy.experimental_distribute_dataset` 和 `tf.distribute.Strategy.distribute_datasets_from_function`：

### `tf.distribute.Strategy.experimental_distribute_dataset`

#### 用法

此 API 将 `tf.data.Dataset` 实例作为输入，并返回 `tf.distribute.DistributedDataset` 实例。您应当使用等于全局批次大小的值对输入数据集进行批处理。此全局批次大小是您要在所有设备中一步处理的样本数。您可以用 Python 样式迭代此分布式数据集，或者使用 `iter` 创建一个迭代器。返回的对象不是 `tf.data.Dataset` 实例，并且不支持以任何方式转换或检查数据集的任何其他 API。如果您没有特定的方式将输入分片到不同副本中，则建议使用此 API。


In [ ]:
global_batch_size = 16
mirrored_strategy = tf.distribute.MirroredStrategy()

dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(100).batch(global_batch_size)
# Distribute input using the `experimental_distribute_dataset`.
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)
# 1 global batch of data fed to the model in 1 step.
print(next(iter(dist_dataset)))

#### 属性

##### 批处理

`tf.distribute` 使用新的批次大小（等于全局批次大小除以同步副本数）对输入 `tf.data.Dataset` 实例进行重新批处理。同步副本数等于训练期间参与梯度全归约的设备数。当用户在分布式迭代器上调用 `next` 时，将在每个副本上返回数据的每个副本批次大小。经过重新批处理的数据集基数将始终为副本数的倍数。下面是一些示例：

- `tf.data.Dataset.range(6).batch(4, drop_remainder=False)`

    - 无分布：

        - 批次 1：[0, 1, 2, 3]
        - 批次 2：[4, 5]

    - 分布在 2 个副本上。最后一个批次 ([4, 5]) 被拆分到 2 个副本中。

    - 批次 1：

        - 副本 1：[0, 1]
        - 副本 2：[2, 3]

    - 批次 2：

        - 副本 2：[4]
        - 副本 2：[5]

- `tf.data.Dataset.range(4).batch(4)`

    - 无分布：
        - 批次 1：[[0], [1], [2], [3]]
    - 分布在 5 个副本上：
        - 批次 1：
            - 副本 1：[0]
            - 副本 2：[1]
            - 副本 3：[2]
            - 副本 4：[3]
            - 副本 5：[]

- `tf.data.Dataset.range(8).batch(4)`

    - 无分布：
        - 批次 1：[0, 1, 2, 3]
        - 批次 2：[4, 5, 6, 7]
    - 分布在 3 个副本上：
        - 批次 1：
            - 副本 1：[0, 1]
            - 副本 2：[2, 3]
            - 副本 3：[]
        - 批次 2：
            - 副本 1：[4, 5]
            - 副本 2：[6, 7]
            - 副本 3：[]

无分布：

批次 1：[0, 1, 2, 3] 

##### 分片

`tf.distribute` 还使用 `MultiWorkerMirroredStrategy` 和 `TPUStrategy` 在多工作进程训练中自动分片输入数据集。每个数据集都是在工作进程的 CPU 设备上创建的。在一组工作进程上自动分片数据集意味着每个工作进程都被分配了整个数据集的一个子集（如果设置了正确的 `tf.data.experimental.AutoShardPolicy`）。这是为了确保在每个步骤中，每个工作进程都将处理非重叠数据集元素的全局批次大小。自动分片有几个不同的选项，可以使用 `tf.data.experimental.DistributeOptions` 来指定。请注意，使用 `ParameterServerStrategy` 的多工作进程训练中没有自动分片，有关使用此策略创建数据集的更多信息，请参阅[参数服务器策略教程](parameter_server_training.ipynb)。 

In [ ]:
dataset = tf.data.Dataset.from_tensors(([1.],[1.])).repeat(64).batch(16)
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
dataset = dataset.with_options(options)

您可以为 `tf.data.experimental.AutoShardPolicy` 设置三个不同的选项：

- AUTO：这是默认选项，意味着将尝试按 FILE 分片。如果未检测到基于文件的数据集，则按 FILE 分片的尝试失败。随后，`tf.distribute` 将退回到按 DATA 分片。请注意，如果输入数据集基于文件，但文件数小于工作进程数，则会引发错误。

- FILE：如果您想将输入文件分片到所有工作进程上，则可以使用此选项。如果输入文件的数量远大于工作进程的数量并且文件中的数据均匀分布，则应使用此选项。如果文件中的数据分布不均匀，则此选项的缺点是有空闲的工作进程。如果文件数量小于工作进程数量，则会引发 `InvalidArgumentError`。如果发生这种情况，请将策略显式设置为 `AutoShardPolicy.DATA`。例如，我们将 2 个文件分布在 2 个工作进程上，每个工作进程有 1 个副本。文件 1 包含 [0, 1, 2, 3, 4, 5]，文件 2 包含 [6, 7, 8, 9, 10, 11]。假设同步的副本总数为 2，全局批次大小为 4。

    - 工作进程 0：
        - 批次 1 =  副本 1：[0, 1]
        - 批次 2 =  副本 1：[2, 3]
        - 批次 3 = 副本 1：[4]
        - 批次 4 = 副本 1：[5]
    - 工作进程 1：
        - 批次 1 = 副本 2：[6, 7]
        - 批次 2 = 副本 2：[8, 9]
        - 批次 3 = 副本 2：[10]
        - 批次 4 = 副本 2：[11]

- DATA：这将在所有工作进程中对元素自动分片。每个工作进程都会读取整个数据集，并且仅处理分配给它的分片。所有其他分片将被丢弃。如果输入文件数小于工作进程数，并且您希望跨所有工作进程对数据更好地分片，通常使用此方法。这种方法的缺点是，将在每个工作进程上读取整个数据集。例如，假设我们将 1 个文件分布到 2 个工作进程中。文件 1 包含 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]。假设同步副本总数为 2。

    - 工作进程 0：
        - 批次 1 =  副本 1：[0, 1]
        - 批次 2 =  副本 1：[4, 5]
        - 批次 3 =  副本 1：[8, 9]
    - 工作进程 1：
        - 批次 1 =  副本 2：[2, 3]
        - 批次 2 =  副本 2：[6, 7]
        - 批次 3 =  副本 2：[10, 11]

- OFF：如果关闭自动分片，则每个工作进程都将处理所有数据。例如，假设我们将 1 个文件分布到 2 个工作进程中。文件 1 包含 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]。假设同步副本总数为 2。那么每个工作器的分布如下：

    - 工作进程 0：

        - 批次 1 =  副本 1：[0, 1]
        - 批次 2 =  副本 1：[2, 3]
        - 批次 3 =  副本 1：[4, 5]
        - 批次 4 =  副本 1：[6, 7]
        - 批次 5 =  副本 1：[8, 9]
        - 批次 6 =  副本 1：[10, 11]

    - 工作进程 1：

        - 批次 1 =  副本 2：[0, 1]
        - 批次 2 =  副本 2：[2, 3]
        - 批次 3 =  副本 2：[4, 5]
        - 批次 4 =  副本 2：[6, 7]
        - 批次 5 =  副本 2：[8, 9]
        - 批次 6 =  副本 2：[10, 11] 

##### 预提取

默认情况下，`tf.distribute` 会向用户提供的 `tf.data.Dataset` 实例末尾添加预提取转换。预提取转换的参数 `buffer_size` 等于同步副本数。

### `tf.distribute.Strategy.distribute_datasets_from_function`

#### 用法

此 API 获取输入函数并返回 `tf.distribute.DistributedDataset` 实例。用户传入的输入函数具有 `tf.distribute.InputContext` 参数，并且应返回 `tf.data.Dataset` 实例。使用此 API，`tf.distribute` 不会对从输入函数返回的用户 `tf.data.Dataset` 实例进行任何进一步的更改。用户负责对数据集进行批处理和分片。`tf.distribute` 在每个工作进程的 CPU 设备上调用输入函数。除了允许用户指定自己的批处理和分片逻辑外，当此 API 用于多工作进程训练时，它还表现出比 `tf.distribute.Strategy.experimental_distribute_dataset` 更出色的可扩展性和性能。

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

def dataset_fn(input_context):
  batch_size = input_context.get_per_replica_batch_size(global_batch_size)
  dataset = tf.data.Dataset.from_tensors(([1.],[1.])).repeat(64).batch(16)
  dataset = dataset.shard(
    input_context.num_input_pipelines, input_context.input_pipeline_id)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(2) # This prefetches 2 batches per device.
  return dataset

dist_dataset = mirrored_strategy.experimental_distribute_datasets_from_function(dataset_fn)

#### 属性

##### 批处理

应当使用每个副本的批次大小对作为输入函数返回值的 `tf.data.Dataset` 实例进行批处理。每个副本的批次大小等于全局批次大小除以参与同步训练的副本数。这是因为 `tf.distribute` 会在每个工作进程的 CPU 设备上调用输入函数。在给定工作进程上创建的数据集应准备好供该工作进程上的所有副本使用。 

##### 分片

`tf.distribute.InputContext` 对象由 `tf.distribute` 在后台创建，作为参数隐式传递到用户的输入函数。它包含有关工作进程数、当前工作进程 ID 等方面的信息。此输入函数可以根据用户使用这些属性（属于 `tf.distribute.InputContext` 对象）设置的策略来处理分片。


##### 预提取

`tf.distribute` 不会向用户提供的输入函数返回的 `tf.data.Dataset` 的末尾添加预提取转换。

注：`tf.distribute.Strategy.experimental_distribute_dataset` 和 `tf.distribute.Strategy.distribute_datasets_from_function` 都会返回不属于 `tf.data.Dataset` 类型的 **`tf.distribute.DistributedDataset` 实例。您可以对这些实例进行迭代（如分布式迭代器部分中所示）并使用 `element_spec` 属性。** 

## 分布式迭代器

与非分布式 `tf.data.Dataset` 实例类似，您将需要在 `tf.distribute.DistributedDataset` 实例上创建一个迭代器以对其进行迭代，并访问 `tf.distribute.DistributedDataset` 中的元素。下面是创建 `tf.distribute.DistributedIterator` 并将其用于训练模型的方式：


### 用法

#### 使用 Python 式 for 循环结构

您可以使用用户友好的 Python 式循环对 `tf.distribute.DistributedDataset` 进行迭代。从 `tf.distribute.DistributedIterator` 返回的元素可以是单个 `tf.Tensor` 或包含每个副本的值的 `tf.distribute.DistributedValues`。将循环放置在 `tf.function` 内有助于提高性能。但是，目前不支持对放置在 `tf.function` 内的 `tf.distribute.DistributedDataset` 的循环使用 `break` 和 `return`。

In [ ]:
global_batch_size = 16
mirrored_strategy = tf.distribute.MirroredStrategy()

dataset = tf.data.Dataset.from_tensors(([1.],[1.])).repeat(100).batch(global_batch_size)
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)

@tf.function
def train_step(inputs):
  features, labels = inputs
  return labels - 0.3 * features

for x in dist_dataset:
  # train_step trains the model using the dataset elements
  loss = mirrored_strategy.run(train_step, args=(x,))
  print("Loss is ", loss)

#### 使用 `iter` 创建显式迭代器

要迭代 `tf.distribute.DistributedDataset` 实例中的元素，您可以在该实例上使用 `iter` API 创建一个 `tf.distribute.DistributedIterator`。使用显式迭代器，您可以迭代固定数量的步骤。为了从 `tf.distribute.DistributedIterator` 实例 `dist_iterator` 获取下一个元素，您可以调用 `next(dist_iterator)`、`dist_iterator.get_next()` 或 `dist_iterator.get_next_as_optional()`。

In [ ]:
num_epochs = 10
steps_per_epoch = 5
for epoch in range(num_epochs):
  dist_iterator = iter(dist_dataset)
  for step in range(steps_per_epoch):
    # train_step trains the model using the dataset elements
    loss = mirrored_strategy.run(train_step, args=(next(dist_iterator),))
    # which is the same as
    # loss = mirrored_strategy.run(train_step, args=(dist_iterator.get_next(),))
    print("Loss is ", loss)

使用 `next()` 或 `tf.distribute.DistributedIterator.get_next()` 时，如果 `tf.distribute.DistributedIterator` 已到达末尾，将引发 OutOfRange 错误。客户端可以在 Python 端捕获该错误，并继续执行其他工作，例如设置检查点和评估。但是，如果您使用的是主机训练循环（即，每个 `tf.function` 运行多个步骤），则此方法无效，如下所示：

```
@tf.function
def train_fn(iterator):
  for _ in tf.range(steps_per_loop):
    strategy.run(step_fn, args=(next(iterator),))
```

`train_fn` 通过将步骤主体包装在 `tf.range` 中来包含多个步骤。在这种情况下，循环中没有依赖项的不同迭代可以并行开始，因此会在先前迭代的计算完成之前在后续的迭代中触发 OutOfRange 错误。一旦引发 OutOfRange 错误，函数中的所有运算都会立即终止。如果您想要避免这种情况，则不引发 OutOfRange 错误的替代方案为 `tf.distribute.DistributedIterator.get_next_as_optional()`。`get_next_as_optional` 返回 `tf.experimental.Optional`，其中包含下一个元素或者不包含任何值（如果 `tf.distribute.DistributedIterator` 已到达末尾）。

In [ ]:
# You can break the loop with get_next_as_optional by checking if the Optional contains value
global_batch_size = 4
steps_per_loop = 5
strategy = tf.distribute.MirroredStrategy(devices=["GPU:0", "CPU:0"])

dataset = tf.data.Dataset.range(9).batch(global_batch_size)
distributed_iterator = iter(strategy.experimental_distribute_dataset(dataset))

@tf.function
def train_fn(distributed_iterator):
  for _ in tf.range(steps_per_loop):
    optional_data = distributed_iterator.get_next_as_optional()
    if not optional_data.has_value():
      break
    per_replica_results = strategy.run(lambda x:x, args=(optional_data.get_value(),))
    tf.print(strategy.experimental_local_results(per_replica_results))
train_fn(distributed_iterator)

## 使用 `element_spec` 属性

如果将分布式数据集的元素传递给 `tf.function` 并且需要 `tf.TypeSpec` 保证，则可以指定 `tf.function` 的 `input_signature` 参数。分布式数据集的输出为 `tf.distribute.DistributedValues`，它可以表示单个设备或多个设备的输入。要获取与此分布式值相对应的 `tf.TypeSpec`，可以使用分布式数据集或分布式迭代器对象的 `element_spec` 属性。

In [ ]:
global_batch_size = 16
epochs = 5
steps_per_epoch = 5
mirrored_strategy = tf.distribute.MirroredStrategy()

dataset = tf.data.Dataset.from_tensors(([1.],[1.])).repeat(100).batch(global_batch_size)
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)

@tf.function(input_signature=[dist_dataset.element_spec])
def train_step(per_replica_inputs):
  def step_fn(inputs):
    return 2 * inputs

  return mirrored_strategy.run(step_fn, args=(per_replica_inputs,))

for _ in range(epochs):
  iterator = iter(dist_dataset)
  for _ in range(steps_per_epoch):
    output = train_step(next(iterator))
    tf.print(output)

## 部分批次

当用户创建的 `tf.data.Dataset` 实例包含的批次大小不能被副本数整除，或者数据集实例的基数不能被批次大小整除时，将遇到部分批次。这意味着，当数据集分布在多个副本上时，某些迭代器上的 `next` 调用将导致 OutOfRangeError。要处理此用例，`tf.distribute` 会在没有更多数据要处理的副本上返回批次大小为 0 的虚拟批次。


对于单工作进程情况，如果迭代器上的 `next` 调用未返回数据，则会创建批次大小为 0 的虚拟批次，并将其与数据集中的实际数据一起使用。在部分批次的情况下，数据的最后一个全局批次将包含实际数据以及虚拟批次数据。现在，用于处理数据的停止条件会检查是否有任何副本具有数据。如果任何副本上都没有数据，则会引发 OutOfRange 错误。

对于多工作进程情况，使用跨副本通信聚合表示每个工作进程上存在数据的布尔值，该布尔值用于标识所有工作进程是否已完成对分布式数据集的处理。由于这涉及跨工作进程通信，因此会涉及一些性能损失。当前，除 `tf.distribute.experimental.MultiWorkerMirroredStrategy` 之外的所有策略均支持此功能。


## 警告

- 除 `tf.distribute.experimental.MultiWorkerMirroredStrategy` 之外的所有策略均支持部分批次。使用 `tf.distribute.experimental.MultiWorkerMirroredStrategy` 时，请确保迭代特定数量的步骤，以便所有副本上存在足够的数据

- `tf.distribute` 当前不支持有状态数据集转换，并且当前将忽略数据集可能具有的任何有状态运算。例如，如果您的数据集包含使用 `tf.random.uniform` 来旋转图像的 `map_fn`，您的数据集计算图将依赖于执行 Python 进程的本地机器上的状态（即，随机种子）。

- `tf.distribute` 当前不支持有状态的数据集转换，并且当前将忽略数据集可能具有的任何有状态运算。例如，如果您的数据集包含使用 `tf.random.uniform` 来旋转图像的 `map_fn`，则您有一个依赖于执行 Python 进程的本地计算机上的状态（即，随机种子）的数据集计算图。

- 请参阅[这篇指南](https://tensorflow.google.cn/guide/data_performance)，了解如何使用 `tf.data` 优化您的输入流水线。一些附加提示：

    - 如果您有多个工作进程并且正在使用 `tf.data.Dataset.list_files` 从匹配一个或多个 glob 模式的所有文件创建数据集，请记住设置 `seed` 参数或设置 `shuffle=False`，这样每个工作进程才能一致地分片文件。

- 如果您的输入流水线包括在记录级别上打乱数据的顺序和解析数据，除非未解析的数据明显大于已解析的数据（通常不是这种情况），否则请先打乱数据，然后再解析，如下面的示例中所示。这样做对内存使用率和性能有利。

```
d = tf.data.Dataset.list_files(pattern, shuffle=False)
d = d.shard(num_workers, worker_index)
d = d.repeat(num_epochs)
d = d.shuffle(shuffle_buffer_size)
d = d.interleave(tf.data.TFRecordDataset,
                 cycle_length=num_readers, block_length=1)
d = d.map(parser_fn, num_parallel_calls=num_map_threads)
```

- `tf.data.Dataset.shuffle(buffer_size, seed=None, reshuffle_each_iteration=None)` 维持 `buffer_size` 元素的内部缓冲区，因此减小 `buffer_size` 可以缓解 OOM 问题。

- 使用 `tf.distribute.experimental_distribute_dataset` 或 `tf.distribute.distribute_datasets_from_function` 时，工作进程处理数据的顺序无法得到保证。如果您使用 `tf.distribute` 来扩展预测，这通常是必需的。但是，您可以为批次中的每个元素插入索引并相应地对输出进行排序。以下代码段是如何对输出进行排序的示例。

注：为方便起见，此处使用 `tf.distribute.MirroredStrategy()`。仅当我们使用多个工作进程并且 `tf.distribute.MirroredStrategy` 用于在单个工作进程上分布训练时，才需要对输入重新排序。

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()
dataset_size = 24
batch_size = 6
dataset = tf.data.Dataset.range(dataset_size).enumerate().batch(batch_size)
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)

def predict(index, inputs):
  outputs = 2 * inputs
  return index, outputs

result = {}
for index, inputs in dist_dataset:
  output_index, outputs = mirrored_strategy.run(predict, args=(index, inputs))
  indices = list(mirrored_strategy.experimental_local_results(output_index))
  rindices = []
  for a in indices:
    rindices.extend(a.numpy())
  outputs = list(mirrored_strategy.experimental_local_results(outputs))
  routputs = []
  for a in outputs:
    routputs.extend(a.numpy())
  for i, value in zip(rindices, routputs):
    result[i] = value

print(result)

<a name="tensorinputs">  # 如果我没有使用规范的 tf.data.Dataset 实例，该如何分布数据？</a>

有时用户无法使用 `tf.data.Dataset` 表示其输入，随后也无法使用上述 API 将数据集分布到多个设备。在这种情况下，您可以使用原始张量或来自生成器的输入。

### 将 experimental_distribute_values_from_function 用于任意张量输入

`strategy.run` 接受 `tf.distribute.DistributedValues`，后者是 `next(iterator)` 的输出。要传递张量值，请使用 `experimental_distribute_values_from_function` 从原始张量构造 `tf.distribute.DistributedValues`。

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()
worker_devices = mirrored_strategy.extended.worker_devices

def value_fn(ctx):
  return tf.constant(1.0)

distributed_values = mirrored_strategy.experimental_distribute_values_from_function(value_fn)
for _ in range(4):
  result = mirrored_strategy.run(lambda x:x, args=(distributed_values,))
  print(result)

### 如果您的输入来自生成器，则使用 tf.data.Dataset.from_generator

如果您具有要使用的生成器函数，则可以使用 `from_generator` API 创建一个 `tf.data.Dataset` 实例。

注：`tf.distribute.TPUStrategy` 当前不支持此功能。

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()
def input_gen():
  while True:
    yield np.random.rand(4)

# use Dataset.from_generator
dataset = tf.data.Dataset.from_generator(
    input_gen, output_types=(tf.float32), output_shapes=tf.TensorShape([4]))
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)
iterator = iter(dist_dataset)
for _ in range(4):
  mirrored_strategy.run(lambda x:x, args=(next(iterator),))